In [1]:
import pandas as pd
from generate_data import get_api_key, gen_cot_prompt

In [2]:
# get the games data

games_df = pd.read_csv("./data/games.csv", index_col = 0)
games_df.head()

,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
id,,,,,,,,,,,,,,,
TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [3]:
games_df.moves[1]

/tmp/ipykernel_176624/1519825386.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  games_df.moves[1]


'd4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4 Bb4+'

# Loading API keys

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("openai_api_key")

## Setting up openai api call environment

In [5]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

In [6]:
games_df.iloc[28]

rated                                                          True
created_at                                          1502780000000.0
last_move_at                                        1502780000000.0
turns                                                            53
victory_status                                               resign
winner                                                        black
increment_code                                                10+10
white_id                                          shivangithegenius
white_rating                                                   1056
black_id                                             biyanivedant01
black_rating                                                   1156
moves             d4 d5 c4 c6 e3 dxc4 Bxc4 Nf6 Nc3 e6 e4 e5 d5 c...
opening_eco                                                     D10
opening_name                                           Slav Defense
opening_ply                                     

In [7]:
moves = games_df.iloc[28]["moves"]
moves

'd4 d5 c4 c6 e3 dxc4 Bxc4 Nf6 Nc3 e6 e4 e5 d5 cxd5 exd5 Bd6 Bg5 O-O Bxf6 Qxf6 Nf3 Bc5 O-O Qg6 Nxe5 Qg5 Nf3 Qh6 Nb5 Na6 d6 Rd8 Qd5 Bxd6 Qxf7+ Kh8 Nxd6 Qxd6 Rfd1 Qf8 Rxd8 Qxd8 h3 Bf5 Qxf5 Qf8 Qd5 Nc7 Qxb7 Qb8 Qd5 Nxd5 Bxd5'

# Testing the prim prompt

# for loop explicit chain of thougt prompting

In [8]:
print("\n".join(gen_cot_prompt(moves)))

White: d4, Black: d5
White: c4, Black: c6
White: e3, Black: dxc4
White: Bxc4, Black: Nf6
White: Nc3, Black: e6
White: e4, Black: e5
White: d5, Black: cxd5
White: exd5, Black: Bd6
White: Bg5, Black: O-O
White: Bxf6, Black: Qxf6
White: Nf3, Black: Bc5
White: O-O, Black: Qg6
White: Nxe5, Black: Qg5
White: Nf3, Black: Qh6
White: Nb5, Black: Na6
White: d6, Black: Rd8
White: Qd5, Black: Bxd6
White: Qxf7+, Black: Kh8
White: Nxd6, Black: Qxd6
White: Rfd1, Black: Qf8
White: Rxd8, Black: Qxd8
White: h3, Black: Bf5
White: Qxf5, Black: Qf8
White: Qd5, Black: Nc7
White: Qxb7, Black: Qb8
White: Qd5, Black: Nxd5
White: Bxd5


In [9]:
system_content_cot = '''Assume you are a chess master.
You will be provided with a list of chess move pairs in Algebraic Notation where 1st move is by White and 2nd by Black and so on.
Your task is to analyse each pair and then generate the rationale behind each White and Black move.
After the pair based analysis you have to write a summary present a list of used strategies during the game and a reason for the winner.
'''
temp = "\n".join(gen_cot_prompt(moves))
user_content_cot = f"Previous move pairs are - \n{temp}"

In [10]:
print(user_content_cot)

Previous move pairs are - 
White: d4, Black: d5
White: c4, Black: c6
White: e3, Black: dxc4
White: Bxc4, Black: Nf6
White: Nc3, Black: e6
White: e4, Black: e5
White: d5, Black: cxd5
White: exd5, Black: Bd6
White: Bg5, Black: O-O
White: Bxf6, Black: Qxf6
White: Nf3, Black: Bc5
White: O-O, Black: Qg6
White: Nxe5, Black: Qg5
White: Nf3, Black: Qh6
White: Nb5, Black: Na6
White: d6, Black: Rd8
White: Qd5, Black: Bxd6
White: Qxf7+, Black: Kh8
White: Nxd6, Black: Qxd6
White: Rfd1, Black: Qf8
White: Rxd8, Black: Qxd8
White: h3, Black: Bf5
White: Qxf5, Black: Qf8
White: Qd5, Black: Nc7
White: Qxb7, Black: Qb8
White: Qd5, Black: Nxd5
White: Bxd5


In [47]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    # messages=[{"role": "user", "content": setup}],
    messages = [
    {
      "role": "system",
      "content": system_content_cot
    },
    {
      "role": "user",
      "content": user_content_cot
    }
  ],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


In the given sequence of moves, the game has progressed as follows:

1. d4, d5: This is a common opening move known as the Queen's Pawn Opening where both players establish control in the center of the board.
2. c4, c6: This move by Black, playing the Slav Defense, aims to defend the pawn on d5 and prepare to develop the knight.
3. e3, dxc4: White captures the c4 pawn, leading to an early exchange.
4. Bxc4, Nf6: White develops the bishop to attack f7 while Black develops the knight to control the center.
5. Nc3, e6: White continues development and prepares to castle kingside, while Black strengthens the center.
6. e4, e5: White advances the e-pawn to gain more central control, while Black retaliates by advancing the e-pawn as well.
7. d5, cxd5: White opens up the center, aiming to create a passed pawn, Black recaptures with the d-pawn.
8. exd5, Bd6: White establishes a strong central pawn duo, Black develops the bishop to put pressure on the center.
9. Bg5, O-O: White pins the f6 knig

In [53]:
def generate_explanation(client : OpenAI, df : pd.DataFrame, sytem_content : str) -> pd.DataFrame:
    return_df = df.copy()

    for i in range(df.shape[0]):
        print(f"[Info] Generating explanation for row {i}")
        moves = df.iloc[i]["moves"]
        id_temp = df.index[i]
        
        user_prompt_move = gen_cot_prompt(moves)
        user_content_cot = f"Previous move pairs are - \n{user_prompt_move}"
        
        stream = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages = [{"role": "system",
                     "content": sytem_content},
                    {"role": "user",
                     "content": user_content_cot}],
        stream=True,)
        
        explanation = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                explanation += chunk.choices[0].delta.content
        return_df.loc[id_temp, "explanation"] = explanation.strip()
    return return_df

In [70]:
games_df

target_df = games_df.iloc[-10:]

In [72]:
games_df[0:10]

,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
id,,,,,,,,,,,,,,,
TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
MsoDV9wj,False,1.504240e+12,1.504240e+12,5,draw,draw,10+0,trelynn17,1250,franklin14532,1002,e4 c5 Nf3 Qa5 a3,B27,Sicilian Defense: Mongoose Variation,4
qwU9rasv,True,1.504230e+12,1.504230e+12,33,resign,white,10+0,capa_jr,1520,daniel_likes_chess,1423,d4 d5 e4 dxe4 Nc3 Nf6 f3 exf3 Nxf3 Nc6 Bb5 a6 ...,D00,Blackmar-Diemer Gambit: Pietrowsky Defense,10
RVN0N3VK,False,1.503680e+12,1.503680e+12,9,resign,black,15+30,daniel_likes_chess,1413,soultego,2108,e4 Nc6 d4 e5 d5 Nce7 c3 Ng6 b4,B00,Nimzowitsch Defense: Kennedy Variation | Link...,5
dwF3DJHO,True,1.503510e+12,1.503510e+12,66,resign,black,15+0,ehabfanri,1439,daniel_likes_chess,1392,e4 e5 Bc4 Nc6 Nf3 Nd4 d3 Nxf3+ Qxf3 Nf6 h3 Bc5...,C50,Italian Game: Schilling-Kostic Gambit,6


In [71]:
target_df

,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
id,,,,,,,,,,,,,,,
dnexZDsv,True,1.499869e+12,1.499870e+12,25,resign,white,10+10,mateuslichess,1252,jamboger,1233,e4 e6 d4 d5 exd5 Qxd5 Nc3 Bb4 a3 Bxc3+ bxc3 Ne...,C01,French Defense: Exchange Variation,5
7IENcPg3,True,1.499815e+12,1.499815e+12,43,mate,white,10+0,jkubb29,1328,jamboger,1252,e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...,C00,French Defense: Two Knights Variation,5
nYOvevdh,True,1.499814e+12,1.499814e+12,9,outoftime,white,10+0,jamboger,1243,yamaguchipolgar,1142,c4 e5 d4 exd4 Qxd4 Nf6 Bg5 Be7 e4,A20,English Opening: King's English Variation,2
uMzb0TPC,True,1.499812e+12,1.499813e+12,58,mate,black,10+10,samael88,1237,jamboger,1231,e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5...,C00,French Defense: Knight Variation,3
EopEqqAa,True,1.499812e+12,1.499812e+12,37,resign,white,10+10,jamboger,1219,samael88,1250,c4 e6 d4 b6 Nc3 Bb7 Nf3 g6 h4 Bg7 Bg5 f6 Bf4 d...,A40,English Defense,4
EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


In [55]:
target_df = generate_explanation(OpenAI(api_key = openai_api_key), target_df, system_content_cot)


[Info] Generating explanation for row 0
[Info] Generating explanation for row 1


In [56]:
target_df

,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,explanation
id,,,,,,,,,,,,,,,,
TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,Analysis of move pairs:\n1. White's move d4: W...
l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4,"Analysis of the move pairs:\n\n1. White: d4, B..."


In [57]:
target_df.iloc[0]['explanation']

"Analysis of move pairs:\n1. White's move d4: White opens with the queen's pawn opening, aiming to control the center of the board and prepare for future development.\n2. Black's move d5: Black responds symmetrically by mirroring White's move, aiming to control the center as well.\n\n3. White's move c4: White plays the Queen's Gambit, offering the c4 pawn to gain control of the center and create potential imbalances in the position.\n4. Black's move c6: Black defends the d5 pawn with the move c6, preparing to develop the knight to c7 and maintain a solid pawn structure.\n\n5. White's move cxd5: White captures the d5 pawn, opening up the c-file and gaining space in the center.\n6. Black's move e6: Black reinforces the center with the e6 move, preparing to fianchetto the dark-squared bishop and maintain a solid pawn structure.\n\n7. White's move dxe6: White captures the e6 pawn, creating a potential weakness in Black's pawn structure and gaining a lead in development.\n8. Black's move fx